# Import Target Dataset

In [114]:
import boto3
from time import sleep
import os
import pandas as pd
import json
import time
import pprint
import numpy as np

## Create Target Time Series File and Validation File

In [115]:
data_dir = 'data'
train_file_name = 'train.csv'
train_data = pd.read_csv(os.path.join(data_dir,train_file_name))
train_df = train_data.copy()
train_df = train_df.rename(columns={'item':'item_id'})

In [116]:
train_df = train_df.set_index('date')
train_df.store = train_df.store.astype(str)
train_df.item_id = train_df.item_id.astype(str)
train_df.index = pd.to_datetime(train_df.index, format = '%Y-%m-%d')
train_df.index = train_df.index.strftime('%Y-%m-%d %H:%M:%S')
cols_order = ['item_id', 'store','sales']
train_df = train_df[cols_order]

train_df.head()

,item_id,store,sales
2013-01-01 00:00:00,1,1,13
2013-01-02 00:00:00,1,1,11
2013-01-03 00:00:00,1,1,14
2013-01-04 00:00:00,1,1,13
2013-01-05 00:00:00,1,1,10


In [117]:
train_df.tail()

,item_id,store,sales
2017-12-27 00:00:00,50,10,63
2017-12-28 00:00:00,50,10,59
2017-12-29 00:00:00,50,10,74
2017-12-30 00:00:00,50,10,62
2017-12-31 00:00:00,50,10,82


## Split into target file and validation file

In [118]:
print(train_df.index.max())
print(train_df.index.min())
start_train_date = '2015-01-01' # Non-inclusive
end_train_date = '2017-12-01' # Non-inclusive
end_val_date = '2018-01-01' # Non-inclusive
stores_sales = train_df.copy()

2017-12-31 00:00:00
2013-01-01 00:00:00


In [119]:
target_stores_sales = stores_sales[stores_sales.index >= start_train_date]
target_stores_sales = target_stores_sales[target_stores_sales.index < end_train_date]
validation_stores_sales = stores_sales[stores_sales.index >= end_train_date]
validation_stores_sales = validation_stores_sales[validation_stores_sales.index < end_val_date]


In [120]:
target_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 532500 entries, 2015-01-01 00:00:00 to 2017-11-30 00:00:00
Data columns (total 3 columns):
item_id    532500 non-null object
store      532500 non-null object
sales      532500 non-null int64
dtypes: int64(1), object(2)
memory usage: 16.3+ MB


In [121]:
validation_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15500 entries, 2017-12-01 00:00:00 to 2017-12-31 00:00:00
Data columns (total 3 columns):
item_id    15500 non-null object
store      15500 non-null object
sales      15500 non-null int64
dtypes: int64(1), object(2)
memory usage: 484.4+ KB


In [122]:
# With the data in a great state, save it off as a CSV
target_time_series_filename = "target_time_series.csv"
target_time_series_path = data_dir + "/" + target_time_series_filename
target_stores_sales.to_csv(target_time_series_path, header=False)

In [123]:
# With the data in a great state, save it off as a CSV
validation_time_series_filename = "validation_time_series.csv"
validation_time_series_path = data_dir + "/" + validation_time_series_filename
validation_stores_sales.to_csv(validation_time_series_path, header=False)

## Parmeters

In [124]:
DATASET_FREQUENCY = "D" 
TIMESTAMP_FORMAT = "yyyy-MM-dd hh:mm:ss"

suffix = str(np.random.uniform())[4:9]


# 원하는 프로젝트 이름을 넣으세요
project = 'StoreItemDemand'
target_suffix = '_target'


target_datasetName= project+'DS' + suffix
target_datasetGroupName= project +'DSG'+ suffix

In [125]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
     data = json.load(notebook_info)
     resource_arn = data['ResourceArn']
     region = resource_arn.split(':')[3]
print(region)

us-east-2


In [126]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')
forecast_query = session.client(service_name='forecastquery')

## Create schema

In [127]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      },
      {
         "AttributeName":"store",
         "AttributeType":"string"
      },       
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      },
   ]
}

## Create Target Time Sereis Dataset

In [128]:
response=forecast.create_dataset(
                    Domain="CUSTOM",
                    DatasetType='TARGET_TIME_SERIES',
                    DatasetName=target_datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema = schema
)

In [129]:
target_datasetArn = response['DatasetArn']
forecast.describe_dataset(DatasetArn=target_datasetArn)

{'DatasetArn': 'arn:aws:forecast:us-east-2:057716757052:dataset/StoreItemDemandDS47645',
 'DatasetName': 'StoreItemDemandDS47645',
 'Domain': 'CUSTOM',
 'DatasetType': 'TARGET_TIME_SERIES',
 'DataFrequency': 'D',
 'Schema': {'Attributes': [{'AttributeName': 'timestamp',
    'AttributeType': 'timestamp'},
   {'AttributeName': 'item_id', 'AttributeType': 'string'},
   {'AttributeName': 'store', 'AttributeType': 'string'},
   {'AttributeName': 'target_value', 'AttributeType': 'float'}]},
 'EncryptionConfig': {},
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2020, 3, 28, 7, 7, 48, 899000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 3, 28, 7, 7, 48, 899000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '5d96ca18-45d7-4819-82ce-cde38e995208',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 28 Mar 2020 07:07:49 GMT',
   'x-amzn-requestid': '5d96ca18-45d7-4819-82ce-cde38e995208',
   'content-len

## Create role
Pleare uncomment if needed

In [130]:
# Use existing role
role_arn = "arn:aws:iam::057716757052:role/WalmartForecast"

In [131]:
# iam = boto3.client("iam")

# # 원하는 Role Name 을 넣으세요
# # role_name = "WalmartForecast" + suffix
# role_name = "WalmartForecast" 
# assume_role_policy_document = {
#     "Version": "2012-10-17",
#     "Statement": [
#         {
#           "Effect": "Allow",
#           "Principal": {
#             "Service": "forecast.amazonaws.com"
#           },
#           "Action": "sts:AssumeRole"
#         }
#     ]
# }

# create_role_response = iam.create_role(
#     RoleName = role_name,
#     AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
# )

# # AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# # if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# # that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
# policy_arn = "arn:aws:iam::aws:policy/AmazonForecastFullAccess"
# iam.attach_role_policy(
#     RoleName = role_name,
#     PolicyArn = policy_arn
# )

# # Now add S3 support
# iam.attach_role_policy(
#     PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
#     RoleName=role_name
# )
# time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

# role_arn = create_role_response["Role"]["Arn"]
# print(role_arn)

## Create a bucket
Uncomment the following code if needed

In [132]:
# Use an existing bucket
bucket_name = "walmart-forecast" 

In [133]:
# print(region)
# s3 = boto3.client('s3')
# account_id = boto3.client('sts').get_caller_identity().get('Account')
# # 원하는 버킷 이름을 넗으세요
# # bucket_name = account_id + "forecastpoc-gsmoon"
# # bucket_name = "walmart-forecast" + suffix
# bucket_name = "walmart-forecast" 
# print(bucket_name)
# if region != "us-east-1":
#     s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
# else:
#     s3.create_bucket(Bucket=bucket_name)

## Create dataset_import_job used to download dataset from S3

In [134]:
# Upload Target File
boto3.Session().resource('s3').Bucket(bucket_name).Object(target_time_series_filename).upload_file(target_time_series_path)
target_s3DataPath = "s3://"+bucket_name+"/"+target_time_series_filename

In [135]:
# Finally we can call import the dataset
datasetImportJobName = 'DSIMPORT_JOB_TARGET_WALMART' + suffix
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=target_datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":target_s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )

In [136]:
ds_target_import_job_arn=ds_import_job_response['DatasetImportJobArn']
print(ds_target_import_job_arn)

arn:aws:forecast:us-east-2:057716757052:dataset-import-job/StoreItemDemandDS47645/DSIMPORT_JOB_TARGET_WALMART47645


In [137]:
%%time

while True:
    dataImportStatus = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_target_import_job_arn)['Status']
    print(dataImportStatus)
    if dataImportStatus != 'ACTIVE' and dataImportStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

CREATE_PENDING
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
ACTIVE
CPU times: user 37.1 ms, sys: 8.08 ms, total: 45.2 ms
Wall time: 5min


In [139]:
%store data_dir
%store project
%store suffix
%store target_suffix
%store region
%store ds_target_import_job_arn
%store target_datasetArn
%store bucket_name
%store role_arn
%store validation_stores_sales

Stored 'data_dir' (str)
Stored 'project' (str)
Stored 'suffix' (str)
Stored 'target_suffix' (str)
Stored 'region' (str)
Stored 'ds_target_import_job_arn' (str)
Stored 'target_datasetArn' (str)
Stored 'bucket_name' (str)
Stored 'role_arn' (str)
Stored 'validation_stores_sales' (DataFrame)
